In [42]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

### preprocess the data

In [43]:
credit = pd.read_csv('./datasets/tmdb_5000_credits.csv')
movie = pd.read_csv('./datasets/tmdb_5000_movies.csv')

In [44]:
movie = movie.merge(credit,on='title') #merge two datasets into one

In [45]:
movie.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [46]:
movie = movie[['id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew']]

In [47]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [48]:
movie.isnull().sum()

id          0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [49]:
movie.dropna(inplace=True) #remove null valued collumn

In [50]:
movie.isnull().sum() #no null values yay!

id          0
title       0
genres      0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

In [51]:
print(movie['genres'][0]) # we have to extract genres from this
type(movie['genres'][0]) 
#we have to convert to list

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]


str

In [52]:
import ast #this is module to convert str to list

def extract(d):
    ans = []
    for i in ast.literal_eval(d):
        ans.append(i['name'])
    return ans

movie['genres'] = movie['genres'].apply(extract)

In [53]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [54]:
movie['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [55]:
movie['keywords'] = movie['keywords'].apply(extract)

In [56]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [57]:
# movie['cast'][0]

In [58]:
def extract2(d):
    ans = []
    cnt = 0
    # we take top 3 character of the movie
    
    for i in ast.literal_eval(d):
        if cnt != 3:
            ans.append(i['name'])
        cnt+=1
    return ans

movie['cast'] = movie['cast'].apply(extract2)
movie['cast'] = movie['cast'].apply(lambda x:x[0:3])

In [59]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [60]:
# movie['crew'][0]

In [61]:
def extract3(d):
    ans = []
    for i in ast.literal_eval(d):
        if i['job'] == 'Director':
            ans.append(i['name'])
    return ans

movie['crew'] = movie['crew'].apply(extract3)

In [62]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [63]:
# removing space from the name avoiding duplicate name
def RemoveSpace(d):
    ans = []
    for i in d:
        ans.append(i.replace(" ",""))
    return ans

In [64]:
movie['cast'] = movie['cast'].apply(RemoveSpace)
movie['crew'] = movie['crew'].apply(RemoveSpace)
movie['genres'] = movie['genres'].apply(RemoveSpace)
movie['keywords'] = movie['keywords'].apply(RemoveSpace)

In [65]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","In the 22nd century, a paraplegic Marine is di...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [66]:
movie['overview'] = movie['overview'].apply(lambda x: x.split())

In [67]:
movie['tags'] = movie['overview'] + movie['genres'] + movie['keywords'] + movie['cast'] + movie['crew']

In [68]:
new = movie.drop(columns=['overview','genres','keywords','cast','crew'])

In [69]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()
# finally... yayyy!!!

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [70]:
new['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [71]:
new['tags'] = new['tags'].apply(lambda x: x.lower())

In [72]:
new['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [73]:
ps = PorterStemmer()

#this function remove same words like lover loving loved and convert to love

def stem(d):
    ans = []
    for i in d.split():
        ans.append(ps.stem(i))
        
    return ' '.join(ans)


In [74]:
new['tags'] = new['tags'].apply(stem)

In [75]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()
#vectorize the tags collumn and remove the stop words of english like 'and', 'is', 'the' etc

In [76]:
vector.shape

(4806, 5000)

In [77]:
vector[0]

array([0, 0, 0, ..., 0, 0, 0])

In [78]:
sim = cosine_similarity(vector)

In [79]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(sim,open('similarity.pkl','wb'))